- https://www.kaggle.com/c/cat-in-the-dat

- EDA
  - https://www.kaggle.com/kabure/eda-feat-engineering-encode-conquer
  - https://www.kaggle.com/werooring/ch7-eda

- BASELINE
  - https://www.kaggle.com/werooring/ch7-baseline


- MODEL MODIFIED
  - https://www.kaggle.com/werooring/ch7-modeling
  - https://www.kaggle.com/werooring/ch7-modeling2


- https://www.kaggle.com/werooring/ch7-notebook

#TYPE: binary classification
#SCORING: ROC AUC
#EDA: feature summary, distribution, binary/nomial/ordinal/date data
#MODEL: logistic regression
#FE: one hot encoding, scaling


In [2]:
import pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

import sklearn
import scipy

import os

In [ ]:
data_path = '/Users/mori/GIT/forLightman/Transcription/TranscriptingMLDL/002cat-feature-encoding/data'
train = pd.read_csv(os.join())


# EDA
- PROCESS: `LOOK AROUND DATA`: FEATRUE SUMMARY → `VISUALIZATION`: TARGET DISTRIBUTION, EXPLAIN VAR DISTRIBUTION → `SUMMARY` & `MODELING STRATEGY`